# Adding and Removing Data
# 데이터 추가하고 제거하기

대부분의 메서드는 새로운 DataFrame 객체를 반환하지만, 일부 메서드는 데이터를 변경시킨다는 것을 알아두자

따라서 원본 데이터를 미리 복사해둔다.

In [6]:
df_copy = df.copy()

## Setup

In [7]:
import pandas as pd

df = pd.read_csv(
    'data/earthquakes.csv', 
    usecols=['time', 'title', 'place', 'magType', 'mag', 'alert', 'tsunami']
)

## Creating new data
### Adding new columns
**변수 할당**과 같은 방식으로 새로운 열을 추가할 수 있다.

모든 데이터가 같은 출처에서 얻은 것이므로 **브로드캐스트**를 사용한 것.

In [8]:
df['source'] = 'USGS API'
df.head()

,alert,mag,magType,place,time,title,tsunami,source
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,USGS API
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,USGS API
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,USGS API


**Boolean mask**로도 **새로운 열추가** 가능

In [9]:
df['mag_negative'] = df.mag < 0 #진도가 음수인지 true, false로
df.head()

,alert,mag,magType,place,time,title,tsunami,source,mag_negative
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,USGS API,False
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,USGS API,False
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,USGS API,False


#### Adding the `parsed_place` column

place 컬럼에는 데이터 일관성이 없다. 같은 개체(entity)에 대해 여러 이름이 존재한다. 

ex) CA, California / NV, Nevada

r', (.*$)' 쉼표 뒤의 문자열의 끝부분까지를 말한다.

In [10]:
df.loc[df.place.str.contains(r'California$'),['place']]

,place
71,"36km ENE of Big Pine, California"
84,"14km NE of East Quincy, California"
96,"60km E of Big Pine, California"
162,"29km ENE of Bridgeport, California"
292,"7km WNW of Tahoe Vista, California"
...,...
8974,"41km ESE of Big Pine, California"
9140,"54km N of Fort Irwin, California"
9162,"14km SSE of Big Pine, California"
9235,"60km E of Big Pine, California"


In [11]:
df.place.str.extract(r', (.*$)') #-> 쉼표 뒷부분의 문자열만 추출해 Series객체를 만든다.

,0
0,CA
1,CA
2,CA
3,CA
4,CA
...,...
9327,CA
9328,CA
9329,Puerto Rico
9330,CA


In [12]:
df.place.str.extract(r', (.*$)')[0] #-> 0컬럼 선택

0                CA
1                CA
2                CA
3                CA
4                CA
           ...     
9327             CA
9328             CA
9329    Puerto Rico
9330             CA
9331             CA
Name: 0, Length: 9332, dtype: object

추출한 문자값들을 정렬하고, 고유한 것만 뽑기

In [13]:
df.place.str.extract(r', (.*$)')[0].sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Australia', 'Azerbaijan', 'B.C., MX', 'Barbuda', 'Bolivia',
       'Bonaire, Saint Eustatius and Saba ', 'British Virgin Islands',
       'Burma', 'CA', 'California', 'Canada', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Ecuador region',
       'El Salvador', 'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala',
       'Haiti', 'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Italy', 'Jamaica', 'Japan', 'Kansas',
       'Kentucky', 'Kyrgyzstan', 'Martinique', 'Mauritius', 'Mayotte',
       'Mexico', 'Missouri', 'Montana', 'NV', 'Nevada', 'New Caledonia',
       'New Hampshire', 'New Mexico', 'New Zealand', 'Nicaragua',
       'North Carolina', 'Northern Mariana Islands', 'Oklahoma', 'Oregon',
       'Pakistan', 'Papua New Guinea', 'Peru', 'Philippines',
       'Puerto Rico', 'Roman

어떤 경우에는 쉼표가 없기도 하고, format이 다 다르기 때문에 
우리의 필요에 맞게 `place` 컬럼을 바꿔준다. 

In [14]:
df['parsed_place'] = df.place.str.replace(
    r'.* of ', '', regex=True # of가 쓰이면서 어떤 것의 어떤 것이라 표현되는 것들 제거.
).str.replace(
    'the ', '' # "the " 제거.
).str.replace(
    r'CA$', 'California', regex=True # fix California
).str.replace(
    r'NV$', 'Nevada', regex=True # fix Nevada
).str.replace(
    r'MX$', 'Mexico', regex=True # fix Mexico
).str.replace(
    r' region$', '', regex=True # region으로 끝나는 부분 삭제.
).str.replace(
    'northern ', '' # remove "northern "
).str.replace(
    'Fiji Islands', 'Fiji' # Fiji islands를 Fiji로 수정.
).str.replace(
    r'^.*, ', '', regex=True # 다른 모든 관계 없는 것 삭제
).str.strip() #여분의 모든 공백을 제거.

완전하진 않지만 place 컬럼이 정리되었다.

In [15]:
df.parsed_place.sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Ascension Island', 'Australia', 'Azerbaijan', 'Balleny Islands',
       'Barbuda', 'Bolivia', 'British Virgin Islands', 'Burma',
       'California', 'Canada', 'Carlsberg Ridge',
       'Central East Pacific Rise', 'Central Mid-Atlantic Ridge', 'Chile',
       'China', 'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'El Salvador',
       'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala', 'Haiti',
       'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indian Ocean Triple Junction', 'Indonesia', 'Iran', 'Iraq',
       'Italy', 'Jamaica', 'Japan', 'Kansas', 'Kentucky',
       'Kermadec Islands', 'Kuril Islands', 'Kyrgyzstan', 'Martinique',
       'Mauritius', 'Mayotte', 'Mexico', 'Mid-Indian Ridge', 'Missouri',
       'Montana', 'Nevada', 'New Caledonia', 'New Hampshire',
       'New Mexico', 'New Zealand', 'Nicaragua', 'North Carolina',


#### Using the `assign()` method to create columns
pandas는 하나의 메서드를 사용해 한 번에 많은 열을 생성할 수 있다. 

`assign()`: **새로운 열을 할당**하는 메서드

in_ca는 california에서 지진이 발생했는지 여부를 알려주고, in_alaska는 alaska에서 지진이 발생했는지 여부를 알려준다.

이 때, **assign() 메서드는 원본 데이터를 바꾸지 않고, 새로운 객체를 반환**한다.

그러므로, 원본 데이터를 수정하고 싶다면, 변수 할당을 사용해 (df = df.assign()....) 저장해야 한다.

In [16]:
df.assign(
    in_ca=df.parsed_place.str.endswith('California'),
    in_alaska=df.parsed_place.str.endswith('Alaska')
).sample(5, random_state=0)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,in_ca,in_alaska
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua",1537749595210,"M 4.8 - 73km SSW of Masachapa, Nicaragua",0,USGS API,False,Nicaragua,False,False
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington",1538227540460,"M 1.1 - 28km NNW of Packwood, Washington",0,USGS API,False,Washington,False,False
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska",1538259609862,"M 1.8 - 77km SSW of Kaktovik, Alaska",0,USGS API,False,Alaska,False,True
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska",1538464751822,"M 1.5 - 102km NW of Arctic Village, Alaska",0,USGS API,False,Alaska,False,True
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA",1538801713880,"M 0.9 - 26km ENE of Pine Valley, CA",0,USGS API,False,California,True,False


assign() 메서드에 lambda 람다 함수를 적용해서 새로운 컬럼을 만들 수 있다.

아래는 in_ca 컬럼과 in_alaska 컬럼 **둘 다 False**이면 **True가 되는 neither 컬럼**을 만든 것이다.

In [17]:
df.assign(
    in_ca=df.parsed_place == 'California',
    in_alaska=df.parsed_place == 'Alaska',
    neither=lambda x: ~x.in_ca & ~x.in_alaska
).sample(5, random_state=0)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,in_ca,in_alaska,neither
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua",1537749595210,"M 4.8 - 73km SSW of Masachapa, Nicaragua",0,USGS API,False,Nicaragua,False,False,True
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington",1538227540460,"M 1.1 - 28km NNW of Packwood, Washington",0,USGS API,False,Washington,False,False,True
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska",1538259609862,"M 1.8 - 77km SSW of Kaktovik, Alaska",0,USGS API,False,Alaska,False,True,False
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska",1538464751822,"M 1.5 - 102km NW of Arctic Village, Alaska",0,USGS API,False,Alaska,False,True,False
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA",1538801713880,"M 0.9 - 26km ENE of Pine Valley, CA",0,USGS API,False,California,True,False,False


## Concatenation 
## DataFrame 합치기

In [18]:
tsunami = df[df.tsunami == 1] #쓰나미가 동반된 지진데이터.
no_tsunami = df[df.tsunami == 0] #쓰나미가 동반되지 않은 지진데이터.

tsunami.shape, no_tsunami.shape

((61, 10), (9271, 10))

row axis를 따라서 연결하는 것이, (`axis=0`) 가 default값. **수평으로 합치기**

In [19]:
pd.concat([tsunami, no_tsunami]).shape

(9332, 10)

`append()` 메서드를 쓰는 것과 동일하다. append()메서드가 concat() 메서드의 래퍼이다. 즉, append() 메서드는 내부적으로 concat() 함수를 호출하여 데이터를 이어붙이는 작업을 수행. **-> concat() 함수가 더 빠르고, 세로로 연결, 가로로 연결 모두 가능.**

In [20]:
tsunami.append(no_tsunami).shape

/var/folders/rc/3jzqjhp56_v2qxb49fkb1x880000gn/T/ipykernel_62846/1630389451.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tsunami.append(no_tsunami).shape


(9332, 10)

tz, felt, ids 등 **빠뜨린 컬럼**들이 있다고 했을 때, 따로 불러와서 axis = 1 열을 기준으로 concat() 

In [21]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California,NaN,",ci37389218,",-480.0
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California,NaN,",ci37389202,",-480.0


concat() 메서드는 index를 기준으로 합치게 된다. 이 때, time 컬럼을 인덱스로 보고 DataFrame을 가져오면 인덱스가 맞지 않아

새로운 행으로 추가하게 된다.

따라서, **행을 추가할 때는 concat과 append**를 이용하지만, 열을 추가할 때는 merge와 join을 이용한다.

In [22]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.4 - 9km NE of Aguanga, CA",0.0,USGS API,False,California,NaN,NaN,NaN
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.3 - 9km NE of Aguanga, CA",0.0,USGS API,False,California,NaN,NaN,NaN
1539475168010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389218,",-480.0
1539475129610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389202,",-480.0


쓰나미 DataFrame와 쓰나미가 동반되지 않은 DataFrame을 합칠 때, 쓰나미가 동반되지 않은 DataFrame에만 type이라는 컬럼이 있다고 했을 때, concat() 메서드의 파라미터로 join을 이용하면 공통된 부분의 데이터만 남고 `inner`, 모든 데이터를 유지한다. `outer`

In [24]:
pd.concat( 
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='outer'
)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,type
36,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,USGS API,False,Christmas Island,NaN
118,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,USGS API,False,Russia,NaN
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California,earthquake
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California,earthquake


인덱스가 의미 없는 경우, `ignore_index` 파라미터를 사용하면 **인덱스가 순차적인 값으로** 매겨진다. -> 인덱스가 그냥 새로 생성됨.

In [15]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place
0,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,USGS API,False,Christmas Island
1,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,USGS API,False,Russia
2,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California
3,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California


## Deleting Unwanted Data
## 원하지 않는 데이터 삭제하기

### del

`del` 을 이용해 삭제. 원본데이터에서 **열**이 삭제된다.

In [46]:
del df['source']
df.columns

Index(['alert', 'mag', 'magType', 'place', 'time', 'title', 'tsunami',
       'mag_negative', 'parsed_place'],
      dtype='object')

삭제 하려는 열이 존재하는지 확실하지 않다면 `try`/`except` 문에 넣는다.

In [47]:
try:
    del df['source']
except KeyError:
    # handle the error here
    print('not there anymore')

not there anymore


### pop

`pop()`을 이용해 열을 삭제할 수도 있다. pop은 삭제하는 열을 반환한다.

In [48]:
mag_negative = df.pop('mag_negative')
df.columns

Index(['alert', 'mag', 'magType', 'place', 'time', 'title', 'tsunami',
       'parsed_place'],
      dtype='object')

`mag_negative` pop()을 통해 반환된 열은 부울 마스크로 쓰일 수 있다.

In [49]:
mag_negative.value_counts()

False    8841
True      491
Name: mag_negative, dtype: int64

Now, we can use `mag_negative` to filter our data:

In [50]:
df[mag_negative].head()

,alert,mag,magType,place,time,title,tsunami,parsed_place
39,NaN,-0.10,ml,"6km NW of Lemmon Valley, Nevada",1539458844506,"M -0.1 - 6km NW of Lemmon Valley, Nevada",0,Nevada
49,NaN,-0.10,ml,"6km NW of Lemmon Valley, Nevada",1539455017464,"M -0.1 - 6km NW of Lemmon Valley, Nevada",0,Nevada
135,NaN,-0.40,ml,"10km SSE of Beatty, Nevada",1539422175717,"M -0.4 - 10km SSE of Beatty, Nevada",0,Nevada
161,NaN,-0.02,md,"20km SSE of Ronan, Montana",1539412475360,"M -0.0 - 20km SSE of Ronan, Montana",0,Montana
198,NaN,-0.20,ml,"60km N of Pahrump, Nevada",1539398340822,"M -0.2 - 60km N of Pahrump, Nevada",0,Nevada


del과 pop을 이용해서 열을 삭제했다면, drop을 이용해서 행과 열을 삭제할 수 있다.

### `drop()`
drop() 메서드는 여러 행이나 열을 제거할 때, 저장하려면 재할당해야한다. (df = df.drop().....) *del이나 pop은 재할당하지 않아도 된다.

**삭제하고 싶은 인덱스의 리스트**를 **drop()** 메서드에 넣는다.

In [52]:
df.drop([0, 1]).head(2)

,alert,mag,magType,place,time,title,tsunami,parsed_place
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,California
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,California


In [54]:
df #원본 df는 그대로

,alert,mag,magType,place,time,title,tsunami,parsed_place
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,California
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,California
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,California
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,California
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,California
...,...,...,...,...,...,...,...,...
9327,NaN,0.62,md,"9km ENE of Mammoth Lakes, CA",1537230228060,"M 0.6 - 9km ENE of Mammoth Lakes, CA",0,California
9328,NaN,1.00,ml,"3km W of Julian, CA",1537230135130,"M 1.0 - 3km W of Julian, CA",0,California
9329,NaN,2.40,md,"35km NNE of Hatillo, Puerto Rico",1537229908180,"M 2.4 - 35km NNE of Hatillo, Puerto Rico",0,Puerto Rico
9330,NaN,1.10,ml,"9km NE of Aguanga, CA",1537229545350,"M 1.1 - 9km NE of Aguanga, CA",0,California


`drop()` 메서드로 열을 삭제하려면 axis = 1을 사용하거나 **columns 인수**를 사용해서 컬럼들의 리스트를 넣어야한다.

In [55]:
cols_to_drop = [
    col for col in df.columns
    if col not in ['alert', 'mag', 'title', 'time', 'tsunami']
]
df.drop(columns=cols_to_drop).head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0


`axis=1`의 방식은 컬럼들의 리스트를 drop함수에 넣어주면 된다.

In [56]:
df.drop(columns=cols_to_drop).equals(
    df.drop(cols_to_drop, axis=1)
)

True

drop() 메서드는 기본 값으로 새로운 DataFrame 객체를 반환하는데, inplace = True를 하면 재할당이 필요없이 바로 원본데이터가 수정된다.

In [57]:
df.drop(columns=cols_to_drop, inplace=True)
df.head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0


In [58]:
df #원본데이터 자체가 변경됨.

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0
...,...,...,...,...,...
9327,NaN,0.62,1537230228060,"M 0.6 - 9km ENE of Mammoth Lakes, CA",0
9328,NaN,1.00,1537230135130,"M 1.0 - 3km W of Julian, CA",0
9329,NaN,2.40,1537229908180,"M 2.4 - 35km NNE of Hatillo, Puerto Rico",0
9330,NaN,1.10,1537229545350,"M 1.1 - 9km NE of Aguanga, CA",0


<hr>

<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="./5-subsetting_data.ipynb">
            <button>&#8592; Previous Notebook</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_02/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_03/1-wide_vs_long.ipynb">
            <button>Chapter 3 &#8594;</button>
        </a>
    </div>
</div>
<hr>